In [1]:
import math
import torch

# Define the two floating-point numbers as tensors
x = torch.tensor(-51.2000)
y = torch.tensor(-26.2062)
z = torch.tensor(2.1944)

# Calculate the arctangent
result_theta = (torch.atan2(y, x) + math.pi) * 180 / math.pi
result_phi = torch.atan2(torch.sqrt(x**2 + y**2), z) * 180 / math.pi
result_r = torch.sqrt(x**2 + y**2 + z**2)

# Print the result
print("theta: ", result_theta)
print("phi: ", result_phi)
print("r: ", result_r)


theta:  tensor(27.1052)
phi:  tensor(87.8151)
r:  tensor(57.5588)


In [2]:
import torch
import torch_geometric
import numpy as np

In [3]:
# Sample data for 5 nodes with 3D positions
# data = torch.tensor([
#     [3, 3, 3],
#     [1.0, 2.5, 3.1],  # Node 1 coordinates (x, y, z)
#     [0.8, 1.7, 4.2],  # Node 2 coordinates
#     [3.4, 0.9, 2.8],  # Node 3 coordinates
#     [2.1, 5.2, 1.5],  # Node 4 coordinates
#     [0.5, 3.8, 6.1],  # Node 5 coordinates
# ])

data = torch.tensor([
    [2.4, 3.6, 1.2],  # Node 1 coordinates (x, y, z)
    [4.9, 4.9, 0],
    [1.4, 2.6, 0.7],  # Node 2 coordinates
    [0.8, 1.4, 0.4],  # Node 3 coordinates
    [1.1, 2.2, 3.5],  # Node 4 coordinates
    [1.5, 2.8, 2.1],  # Node 5 coordinates
])


data.pos = data  # Assuming our data object already holds the positions

# Assuming two separate graphs (components)
# data.batch = torch.tensor([0, 0, 1, 1, 0])
data.batch = torch.tensor([0, 0, 1, 1, 0, 0])


# batch = data.batch.view(-1, 1).repeat(1, 3)  # Repeat batch values 3 times for each node

# pos = torch.cat([data, batch], dim=-1)

# print(data.shape)

# Voxel size of 2 in all dimensions (cubic voxels)
voxel_size = (2, 2, 2)
# voxel_size = (2, 2, 3)
# Define minimum coordinates (-2, 1, 0)
start = (0, 0, 0)

# Define maximum coordinates (5, 8, 7)
end = (8, 8, 8)

cluster = torch_geometric.nn.pool.voxel_grid(pos=data, batch=data.batch, start=start, end=end, size=voxel_size)
# cluster = torch_geometric.nn.pool.voxel_grid(pos=data, batch=data.batch, size=voxel_size)

cluster

tensor([  6,  12, 130, 125,  30,  30])

In [4]:
data, start[0], data[0][0*3+0], data[0][0*3+0] - start[0], voxel_size

(tensor([[2.4000, 3.6000, 1.2000],
         [4.9000, 4.9000, 0.0000],
         [1.4000, 2.6000, 0.7000],
         [0.8000, 1.4000, 0.4000],
         [1.1000, 2.2000, 3.5000],
         [1.5000, 2.8000, 2.1000]]),
 0,
 tensor(2.4000),
 tensor(2.4000),
 (2, 2, 2))

In [21]:
def check_point_in_ellipsoid(pos, center, size, c_0):
    c_0 = int(c_0)
    x, y, z = pos[0], pos[1], pos[2]
    x_0, y_0, z_0 = center[0], center[1], center[2]
    a, b, c = size[0], size[1], size[2]
    # Calculate the equation of the ellipsoid
    equation = ((x-x_0)**2 / a**2) + ((y-y_0)**2 / b**2) + ((z-z_0)**2 / c**2)

    # Check if the point is inside, on, or outside the ellipsoid
    if equation <= 1:
        c_0 =  c_0
    else:
        if x - a >= 0 and y - b >= 0 and z - c >= 0:    # 5
            # return "Outside Right Top Front"
            c_0 = c_0 + 4
        elif x - a >= 0 and y - b >= 0 and z - c < 0:   # 9
            # return "Outside Right Top Back"
            c_0 = c_0 + 8
        elif x - a >= 0 and y - b < 0 and z - c >= 0:   # 3
            # return "Outside Right Bottom Front"
            c_0 = c_0 + 2
        elif x - a >= 0 and y - b < 0 and z - c < 0:    # 7
            # return "Outside Right Bottom Back"
            c_0 = c_0 + 6
        elif x - a < 0 and y - b >= 0 and z - c >= 0:   # 4
            # return "Outside Left Top Front"
            c_0 = c_0 + 3
        elif x - a < 0 and y - b >= 0 and z - c < 0:    # 8
            # return "Outside Left Top Back"
            c_0 = c_0 + 7
        elif x - a < 0 and y - b < 0 and z - c >= 0:    # 2
            # return "Outside Left Bottom Front"
            c_0 = c_0 + 1
        elif x - a < 0 and y - b < 0 and z - c < 0:     # 6
            # return "Outside Left Bottom Back"
            c_0 = c_0 + 5
        else:
            c_0 = c_0
            
    return torch.tensor(c_0)

In [22]:
def grid_kernel_python_new(pos, size, start, end, D, numel):
  out = [0]
  thread_idx = 0
  px = py = pz = 0
  c = 0
  cx = cy = cz = 0
  k = 1
  kx = ky = kz = 1
  center = [0, 0, 0]
  for d in range(D):
    # print("c_start: ", c)
    # print("k_start: ", k)
    p = pos[thread_idx*D + d] - start[d]
    steps = p // size[d]
    center[d] = float(steps * size[d] + size[d]/2)
    c += steps * k
    k *= ((end[d] - start[d]) // size[d])
    
    out[thread_idx] = c * 9
    out[thread_idx] = check_point_in_ellipsoid(pos, center, size, out[thread_idx])
    # print("c_end: ", c)
    # print("k_end: ", k)

    if d == 0:
      px = float(p)
      cx = int(c)
      kx = int(k)
      # print("cx: ", cx)

    elif d == 1:
      py = float(p)
      cy = int(c)
      ky = int(k)
      # print("cy: ", cy)

    elif d == 2:
      pz = float(p)
      cz = int(c)
      kz = int(k)
      # print("cz: ", cz)

  print("pos: ", pos)
  print("cx: ", cx, "kx: ", kx, "px: ", px)
  print("cy: ", cy, "ky: ", ky, "py: ", py)
  print("cz: ", cz, "kz: ", kz, "pz: ", pz)
  print("center: ", center)

  return out

for i in range(len(data)):
  indices = grid_kernel_python_new(data[i], voxel_size, start, end, len(data[0]), 1000)
  print(indices, type(indices))

pos:  tensor([2.4000, 3.6000, 1.2000])
cx:  1 kx:  4 px:  2.4000000953674316
cy:  5 ky:  16 py:  3.5999999046325684
cz:  5 kz:  64 pz:  1.2000000476837158
center:  [3.0, 3.0, 1.0]
[tensor(45)] <class 'list'>
pos:  tensor([4.9000, 4.9000, 0.0000])
cx:  2 kx:  4 px:  4.900000095367432
cy:  10 ky:  16 py:  4.900000095367432
cz:  10 kz:  64 pz:  0.0
center:  [5.0, 5.0, 1.0]
[tensor(90)] <class 'list'>
pos:  tensor([1.4000, 2.6000, 0.7000])
cx:  0 kx:  4 px:  1.399999976158142
cy:  4 ky:  16 py:  2.5999999046325684
cz:  4 kz:  64 pz:  0.699999988079071
center:  [1.0, 3.0, 1.0]
[tensor(36)] <class 'list'>
pos:  tensor([0.8000, 1.4000, 0.4000])
cx:  0 kx:  4 px:  0.800000011920929
cy:  0 ky:  16 py:  1.399999976158142
cz:  0 kz:  64 pz:  0.4000000059604645
center:  [1.0, 1.0, 1.0]
[tensor(0)] <class 'list'>
pos:  tensor([1.1000, 2.2000, 3.5000])
cx:  0 kx:  4 px:  1.100000023841858
cy:  4 ky:  16 py:  2.200000047683716
cz:  20 kz:  64 pz:  3.5
center:  [1.0, 3.0, 3.0]
[tensor(180)] <class 'li

In [20]:
pos = torch.tensor([
    [4.4, 5.6, 2.2],
    [4.9, 4.9, 0],
    [1.4, 2.6, 0.7], 
    [0.8, 1.4, 0.4], 
    [1.1, 2.2, 3.5],  
    [1.5, 2.8, 2.1],  
])

center = torch.Tensor([
    [3.0, 3.0, 1.0],
    [5.0, 5.0, 1.0],
    [1.0, 3.0, 1.0],
    [1.0, 1.0, 1.0],
    [1.0, 3.0, 3.0],
    [1.0, 3.0, 3.0]
])

size = torch.Tensor([2.0, 2.0, 2.0]) 

c_0 = torch.tensor(45.)

new_c_0 = check_point_in_ellipsoid(pos[0], center[0], size, c_0)
new_c_0

C:\Users\User\AppData\Local\Temp\ipykernel_16256\81980031.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(c_0)


tensor(49.)

In [85]:
def grid_kernel_python(pos, size, start, end, D, numel):
  out = [0]
  thread_idx = 0
  c = 0
  k = 1
  for d in range(D):
    # print("c_start: ", c)
    # print("k: ", k)
    p = pos[thread_idx*D + d] - start[d]
    c += (p // size[d]) * k
    k *= ((end[d] - start[d]) // size[d]+ 1) 
  
    out[thread_idx] = c
    # print("c_end: ", c)

  return out

for i in range(5):
  indices = grid_kernel_python(data[i], voxel_size, start, end, len(data[i]), 1000)
  print(indices)


[tensor(17.)]
[tensor(33.)]
[tensor(14.)]
[tensor(10.)]
[tensor(53.)]


In [70]:
grid_extent_x = (end[0] - start[0]) / voxel_size
grid_extent_y = (end[1] - start[1]) / voxel_size
grid_extent_z = (end[2] - start[2]) / voxel_size

total_voxels = int(grid_extent_x * grid_extent_y * grid_extent_z)
print("Total number of voxels:", total_voxels)


Total number of voxels: 42


In [72]:
def try_this(pos, batch, start, end, size):
    print(pos.dim())

In [52]:
# Sample data for 5 nodes with 3D positions
data = torch.tensor([
    [1.0, 2.5, 3.1],  # Node 1 coordinates (x, y, z)
    [0.8, 1.7, 4.2],  # Node 2 coordinates
    [3.4, 0.9, 2.8],  # Node 3 coordinates
    [2.1, 5.2, 1.5],  # Node 4 coordinates
    [0.5, 3.8, 6.1],  # Node 5 coordinates
])

data.pos = data  # Assuming our data object already holds the positions

# Assuming two separate graphs (components)
data.batch = torch.tensor([0, 0, 1, 1, 0])


# batch = data.batch.view(-1, 1).repeat(1, 3)  # Repeat batch values 3 times for each node

# pos = torch.cat([data, batch], dim=-1)

# print(data.shape)

# Voxel size of 2 in all dimensions (cubic voxels)
voxel_size = 2
# voxel_size = (2, 2, 3)
# Define minimum coordinates (-2, 1, 0)
start = (-2, 1, 0)

# Define maximum coordinates (5, 8, 7)
end = (5, 8, 7)

# cluster = torch_geometric.nn.pool.voxel_grid(pos=data, batch=data.batch, start=start, end=end, size=voxel_size)
# cluster = torch_geometric.nn.pool.voxel_grid(pos=data, batch=data.batch, size=voxel_size)
dim = data.size(1)
'''data.size()   # 5,3 -> data.size(1) => 3'''

data = torch.cat([data, data.batch.view(-1, 1).to(data.dtype)], dim=-1) # Add the batch value to each node dim = 5x4
'''
data.dtype  # torch.float32
data.device # 'cpu'
'''
voxel_size = torch.tensor(voxel_size, dtype=data.dtype, device=data.device)
'''voxel_size  #tensor(2.)'''
voxel_size = torch_geometric.utils.repeat.repeat(voxel_size, dim)  # Repeat the voxel size for each dimension -> 4 
voxel_size = torch.cat([voxel_size, voxel_size.new_ones(1)])   # tensor([2., 2., 2., 1.])


start = torch.tensor(start, dtype=data.dtype, device=data.device)   # tensor([-2.,  1.,  0.])
start = torch_geometric.utils.repeat.repeat(start, dim)             # tensor([-2.,  1.,  0.])
start = torch.cat([start, start.new_zeros(1)])                      # tensor([-2.,  1.,  0.,  0.])


end = torch.tensor(end, dtype=data.dtype, device=data.device)       # tensor([5., 8., 7.])
end = torch_geometric.utils.repeat.repeat(end, dim)                 # tensor([5., 8., 7.])
end = torch.cat([end, end.new_ones(1)])                             # tensor([5., 8., 7., 1.])

cluster = torch.ops.torch_cluster.grid(data, voxel_size, start, end)          # tensor([17, 33, 82, 74, 53])
cluster

tensor([17, 33, 82, 74, 53])

In [31]:
cluster = torch.tensor([78, 45, 45, 23, 89])

'''
unique = tensor([23, 45, 78, 89])
cluster = tensor([2, 1, 1, 0, 3])
'''

'\nunique = tensor([23, 45, 78, 89])\ncluster = tensor([2, 1, 1, 0, 3])\n'

In [32]:
unique, cluster = torch.unique(cluster, sorted=True, return_inverse=True)   # tensor([17, 33, 53, 74, 82]) tensor([0, 1, 4, 3, 2])
unique, cluster, counts = torch.unique(cluster, sorted=True, return_inverse=True, return_counts=True)   # tensor([0, 1, 2, 3, 4]) tensor([0, 1, 1, 2, 3]) tensor([1, 2, 1, 1])
cluster, counts.max().item(), counts

(tensor([2, 1, 1, 0, 3]), 2, tensor([1, 2, 1, 1]))

In [44]:
n = unique.shape[0]     # 5
k = counts.max().item() # < 5
p2v_map = cluster.new_zeros(n, k)
# counts.unsqueeze(-1)    # tensor([[1], [1], [1], [1], [1]])
# torch.arange(k).unsqueeze(0)  # tensor([[0]])
mask = torch.arange(k).unsqueeze(0) < counts.unsqueeze(-1)  # tensor([[True], [True], [True], [True], [True]])
p2v_map[mask] = torch.argsort(cluster)  # indices are sorted in ascending order
# cluster
torch.argsort(cluster)
# cluster, p2v_map, counts

tensor([3, 1, 2, 0, 4])

In [50]:
v2p_map, k, counts = cluster, p2v_map, counts
n = counts.shape[0]  # 5
N = v2p_map.shape[0] # 5
n_max = k            # tensor([[0], [1], [4], [3], [2]])
v2p_map, sort_idx = v2p_map.sort()       # torch.return_types.sort(values=tensor([0, 1, 2, 3, 4]), indices=tensor([0, 1, 4, 3, 2]))
sort_idx

tensor([0, 1, 4, 3, 2])

5